In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetescsv/diabetes.csv


**Loading the dataset**

In [2]:
df=pd.read_csv("/kaggle/input/diabetescsv/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Separating input and output features

In [4]:
X=df.drop(columns=["Outcome"])
y=df["Outcome"]

In [5]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [6]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

**Standardizing data**

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [9]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Defining the Network Architechture**

In [11]:
import torch.nn as nn
import torch.nn.functional as F

In [12]:
class Model(nn.Module):
  def __init__(self,input_feat=8,hidden1=18,hidden2=19,output_feat=1):
    super(Model,self).__init__()
    self.fc1=nn.Linear(input_feat,hidden1)
    self.fc2=nn.Linear(hidden1,hidden2)
    self.output=nn.Linear(hidden2,output_feat)

  def forward(self,x): # this methods define the forward pass of the neural network
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.sigmoid(self.output(x))
    return x

In [15]:
model=Model()

In [16]:
import torch
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
epochs = 1000
for epoch in range(epochs):
    # Convert arrays to tensors
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train, dtype=torch.float32)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 0.6680
Epoch [20/1000], Loss: 0.6548
Epoch [30/1000], Loss: 0.6392
Epoch [40/1000], Loss: 0.6203
Epoch [50/1000], Loss: 0.5979
Epoch [60/1000], Loss: 0.5730
Epoch [70/1000], Loss: 0.5471
Epoch [80/1000], Loss: 0.5227
Epoch [90/1000], Loss: 0.5024
Epoch [100/1000], Loss: 0.4866
Epoch [110/1000], Loss: 0.4758
Epoch [120/1000], Loss: 0.4686
Epoch [130/1000], Loss: 0.4635
Epoch [140/1000], Loss: 0.4593
Epoch [150/1000], Loss: 0.4556
Epoch [160/1000], Loss: 0.4521
Epoch [170/1000], Loss: 0.4487
Epoch [180/1000], Loss: 0.4454
Epoch [190/1000], Loss: 0.4418
Epoch [200/1000], Loss: 0.4382
Epoch [210/1000], Loss: 0.4346
Epoch [220/1000], Loss: 0.4312
Epoch [230/1000], Loss: 0.4278
Epoch [240/1000], Loss: 0.4244
Epoch [250/1000], Loss: 0.4211
Epoch [260/1000], Loss: 0.4177
Epoch [270/1000], Loss: 0.4142
Epoch [280/1000], Loss: 0.4108
Epoch [290/1000], Loss: 0.4074
Epoch [300/1000], Loss: 0.4041
Epoch [310/1000], Loss: 0.4009
Epoch [320/1000], Loss: 0.3977
Epoch [330/1000],

In [21]:
y_test

661    1
122    0
113    0
14     1
529    0
      ..
476    1
482    0
230    1
527    0
380    0
Name: Outcome, Length: 154, dtype: int64

In [23]:
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)


In [24]:
with torch.no_grad():
    y_predicted = model(torch.tensor(X_test, dtype=torch.float32))
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(torch.tensor(y_test).unsqueeze(1)).sum() / float(y_test.shape[0])
    print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7532


/tmp/ipykernel_33/1848401176.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc = y_predicted_cls.eq(torch.tensor(y_test).unsqueeze(1)).sum() / float(y_test.shape[0])
